In [1]:
import pandas as pd
import numpy
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Data

The data was donated on November 5, 2015 to University of California Irvine Machine Learning repository and can be found at the URL link `https://archive.ics.uci.edu/dataset/352/online+retail`.

It's stated in the documentation that this data is compiled of transactions occurring between January 12, 2010 to September 12, 2011 for UK-based and registered non-store online retail with 541,909 instances with no missing values.

In [2]:
# Read Excel data
data = pd.read_excel('./Data/online+retail/Online Retail.xlsx',
                    header = 0)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [4]:
data.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [5]:
data.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


- Dates seem to be from December 1, 2010 to December 9, 2011
- There do seem to be missing/null values in the `Description` and `CustomerID` features

# Data Cleaning

Although the documentation explicitly states there are no missing values, there do seem to be values missing in the `Description` and `CustomerID` features.
I want to investigate to see if they can be populated

## Missing/Multiple `Description`
My first instinct is to use the `StockCode` from other instances in the data to populate the missing `Description`values. First I want to isolate for instances where the `Description` is null/missing

In [6]:
# Isolate null Descriptions
null_des = data[data['Description'].isnull()]

Before moving forward, we need to figure out if there is a 1-to-1 relationship between the `StockCode` and `Description` features.

In [7]:
# Use the groupby function to get the number of unique Descriptions by StockCode
stock_code_desc_nunique = data.groupby('StockCode')[['Description']].nunique()

In [8]:
stock_code_desc_nunique[stock_code_desc_nunique['Description'] > 1].sort_values('Description', ascending = False)

,Description
StockCode,
20713,8
23084,7
85175,6
21830,6
21181,5
...,...
22892,2
22896,2
22900,2


It does appear there are 650 `StockCode` that have at more than 1 unique `Description`. The next question is if the descriptions are similar or related in anyway and does capitalizing all the letters in the description change anything.

In [9]:
# Are there any similarities in the "extraneous" descriptions?
data[data['StockCode'].isin([20713,23084,85175,21830,21181])]['Description'].unique()

array(['JUMBO BAG OWLS', 'PLEASE ONE PERSON METAL SIGN',
       'ASSORTED CREEPY CRAWLIES', 'CACTI T-LIGHT CANDLES',
       'Amazon sold sets', 'dotcom sold sets', 'wrongly sold sets',
       '? sold as sets?', nan, 'MERCHANT CHANDLER CREDIT ERROR, STO',
       'RABBIT NIGHT LIGHT', 'on cargo order',
       'wrongly marked. 23343 in box', 'sold as 1', '?',
       'temp adjustment', 'allocate stock for dotcom orders ta',
       'add stock to allocate online orders', 'for online retail orders',
       'Amazon', 'damaged', 'OOPS ! adjustment', 'website fixed',
       'wrongly coded-23343', 'found', 'Found', 'wrongly marked 23343',
       'Marked as 23343', 'wrongly coded 23343', 'adjustment', 'check',
       'dotcom'], dtype=object)

In [10]:
# How do the numbers change when we capitalize all the descriptions?
data['Description'] = data['Description'].str.upper()

In [11]:
# Use the groupby function to get the number of unique Descriptions by StockCode
stock_code_desc_nunique_upper = data.groupby('StockCode')[['Description']].nunique()

In [12]:
stock_code_desc_nunique_upper[stock_code_desc_nunique_upper['Description'] > 1].sort_values('Description', ascending = False)

,Description
StockCode,
20713,7
23084,7
21830,6
85175,6
72807A,5
...,...
22892,2
22896,2
22900,2


By capitalizing all the letters in the descriptions, it didn't change the total number of `StockCode` that have more than 1 unique `Description`. The description themselves are not standardized to follow a distinguished convention and are simply user input, therefore cleaning up the descriptions may not be time efficient.

I do want to investigate what is happening to the descriptions that contain "wrong" and "found".

In [13]:
wrong_desc = data[(data['Description'].str.contains('WRONG'))
                    & (data['Description'].notnull())]

wrong_desc

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
114522,546018,85172,WRONGLY SOLD AS SETS,-600,2011-03-08 17:23:00,0.0,NaN,United Kingdom
114538,546023,85175,WRONGLY SOLD SETS,-975,2011-03-08 17:29:00,0.0,NaN,United Kingdom
117894,546407,22719,WRONG BARCODE (22467),-178,2011-03-11 16:24:00,0.0,NaN,United Kingdom
117895,546408,22467,WRONGLY SOLD (22719) BARCODE,170,2011-03-11 16:25:00,0.0,NaN,United Kingdom
128464,547336,21689,WRONG BARCODE,-323,2011-03-22 11:45:00,0.0,NaN,United Kingdom
168324,551019,22467,WRONG CODE?,-100,2011-04-26 11:19:00,0.0,NaN,United Kingdom
168325,551020,22719,WRONG CODE,-110,2011-04-26 11:19:00,0.0,NaN,United Kingdom
263884,560039,20713,WRONGLY MARKED. 23343 IN BOX,-3100,2011-07-14 14:27:00,0.0,NaN,United Kingdom
277345,561103,85103,STOCK CREDITTED WRONGLY,-32,2011-07-25 11:23:00,0.0,NaN,United Kingdom
380687,569830,23343,WRONGLY CODED 20713,800,2011-10-06 12:38:00,0.0,NaN,United Kingdom


Descriptions that contain the word "WRONG" seem to be clerical/operational errors. Each instance seems to have corresponding return/refund and re-purchase/correction quantities which should balance each other out when doing further analysis.

In [14]:
found_desc = data[(data['Description'].str.contains('FOUND'))
                    & (data['Description'].notnull())]

found_desc

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
39047,539611,85135B,FOUND,53,2010-12-20 14:33:00,0.0,NaN,United Kingdom
51757,540673,21644,FOUND,144,2011-01-10 16:04:00,0.0,NaN,United Kingdom
51758,540674,22837,FOUND,26,2011-01-10 16:05:00,0.0,NaN,United Kingdom
51759,540675,20748,FOUND,40,2011-01-10 16:05:00,0.0,NaN,United Kingdom
132630,547673,47593A,FOUND,13,2011-03-24 13:57:00,0.0,NaN,United Kingdom
166645,550950,17091A,FOUND,84,2011-04-21 16:14:00,0.0,NaN,United Kingdom
170538,551324,85017C,FOUND,13,2011-04-27 15:47:00,0.0,NaN,United Kingdom
228879,556963,35965,FOUND SOME MORE ON SHELF,-9,2011-06-16 09:40:00,0.0,NaN,United Kingdom
241840,558245,22734,FOUND,8,2011-06-27 16:28:00,0.0,NaN,United Kingdom
242709,558369,21082,FOUND,3,2011-06-28 15:40:00,0.0,NaN,United Kingdom


"Found" instances are a bit more interesting, especially the instance where -9 were "found". I would still like to have each StockCode to have their own unique Description. Therefore, I may create a `CustomerID` unique to found items to offset changing the "Found" description.

Before assuming the only odd instances of the `Description` contain "wrong", "found" or null values, we should take a look at how many `StockCode` would have multiple descriptions

In [15]:
# Remove items that have found, wrong, and blank descriptions
maybe_clean_desc = data[(~data['Description'].str.contains('FOUND', na = False))
                        & ~(data['Description'].str.contains('WRONG', na = False))
                        & (data['Description'].notnull())]\
                    .groupby('StockCode')\
                    [['Description']]\
                    .nunique()

In [16]:
# How many unique items have more than one description now?
maybe_clean_desc[maybe_clean_desc['Description'] > 1]\
    .sort_values('Description',
                 ascending = False)

,Description
StockCode,
23084,7
21830,6
21181,5
23131,5
72807A,5
...,...
22785,2
22784,2
22777,2


In [17]:
# How do the top 5 offenders descriptions look?
data[data['StockCode'].isin([23084,21830,21181,23131,'72807A'])]['Description'].unique()

array(['SET/3 ROSE CANDLE IN JEWELLED BOX',
       'PLEASE ONE PERSON METAL SIGN', 'ASSORTED CREEPY CRAWLIES',
       'MERCHANT CHANDLER CREDIT ERROR, STO',
       'MISTLETOE HEART WREATH CREAM', 'RABBIT NIGHT LIGHT',
       'MISELTOE HEART WREATH WHITE', 'ON CARGO ORDER', nan,
       'MISELTOE HEART WREATH CREAM', 'SOLD AS 1', '?', 'TEMP ADJUSTMENT',
       'ALLOCATE STOCK FOR DOTCOM ORDERS TA',
       'ADD STOCK TO ALLOCATE ONLINE ORDERS', 'FOR ONLINE RETAIL ORDERS',
       'AMAZON', 'DAMAGED', 'OOPS ! ADJUSTMENT', 'WEBSITE FIXED',
       'ADJUSTMENT', 'CHECK', 'HAD BEEN PUT ASIDE', 'DOTCOM',
       'WET PALLET', 'DAMAGES', '???MISSING'], dtype=object)

When removing items that don't contain "wrong", "found" or null, 634 items still have more than one unique description. That's likely due to inconsistent user inputs, spelling errors, and nulls. When doing further analysis, it should be noted to rely on the `StockCode` when distinguishing between different items, at least until a more reliable item key can be established.
___

# Null `CustomerID`

Next, I'd like to see if there are any patterns within the data that are correlated to missing `CustomerID` values. 

In [18]:
missing_custom = data[data['CustomerID'].isna()]

In [19]:
missing_custom

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


In [27]:
print('Percent of Invoice Numbers with missing Customer ID: {:.2f}'.format(missing_custom['InvoiceNo'].nunique()/data['InvoiceNo'].nunique()))

Percent of Invoice Numbers with missing Customer ID: 0.14


In [28]:
print('Percent of Invoice Dates with missing Customer ID: {:.2f}'.format(missing_custom['InvoiceDate'].nunique()/data['InvoiceDate'].nunique()))

Percent of Invoice Dates with missing Customer ID: 0.14


In [31]:
print('Percent of StockCodes with missing Customer ID: {:.2f}'.format(missing_custom['StockCode'].nunique()/data['StockCode'].nunique()))

Percent of StockCodes with missing Customer ID: 0.94


Instances where the customer ID is missing doesn't seem to be specific to any particular `StockCode`, meaning the orders affects most items in stock and could possibly be clerical corrections, import/export of inventory across vendors/suppliers, or internal company transactions.

The lower percentage of Invoice Numbers and Invoice Dates could allude to more unique instances or occasional corrections, however, it may be impossible to determine a Customer ID for each one.

# Export Data for Analysis

In [32]:
data.to_excel('./Data/Online Retail_clean.xlsx',
              index = False)

# Problem
- Clean and preprocess
- Perform RFM analysis (Recency, Frequency, Monetary)
- Create cohort retention charts (month-to-moth repeat rate)
- Build a Tableau dashboard that lets stakeholders see which customer segments are most profitable